In [2]:
import pandas as pd

path = r"C:\Users\z0040jeb\Desktop\predictions\ON_one_year\ON_performance_report_25122024_1507_table_True_onlytable_False.xlsx"

df = pd.read_excel(path,usecols=lambda column: column not in ['Unnamed: 0']).reset_index(drop=True)

df.head(3)

,Date,Open,High,Low,Close,Volume,Close_EMA_13,Close_EMA_8,Predicted_High,Predicted_Low,...,Profitable_trades_(%),Total_win_trades,cumulative_profit,High_Mean_Negative_Deviation,Low_Mean_Negative_Deviation,model,total_tokens,system_fingerprint,combined_score,trade_tracking
0,2024-12-23,65.50,67.92,65.45,67.90,8302204,67.900000,67.900000,72.000,66.00,...,48,24,0.000,-2.286286,-1.553271,gpt-4o-2024-05-13,13311,fp_20cb129c3a,0.146062,NaN
1,2024-12-16,65.40,70.58,63.94,65.50,44152334,67.557143,67.366667,69.715,64.32,...,48,24,660.195,-2.286286,-1.553271,gpt-4o-2024-05-13,13311,fp_20cb129c3a,0.146062,1.0
2,2024-12-09,66.52,68.66,64.80,65.96,26548753,67.328980,67.054074,71.300,64.59,...,48,24,660.195,-2.286286,-1.553271,gpt-4o-2024-05-13,13311,fp_20cb129c3a,0.146062,NaN


In [10]:
df['difference_open_predicted_high'] = df.apply(lambda x: x['Predicted_High']-x['Open'],axis=1)
df['difference_open_predicted_low'] = df.apply(lambda x: -x['Predicted_Low']+x['Open'],axis=1)
df.head(3)


,Date,Open,High,Low,Close,Volume,Close_EMA_13,Close_EMA_8,Predicted_High,Predicted_Low,...,total_tokens,system_fingerprint,combined_score,trade_tracking,difference_open_predicted,predicted_diff,open_bin,diff_bin,difference_open_predicted_low,difference_open_predicted_high
0,2024-12-23,65.50,67.92,65.45,67.90,8302204,67.900000,67.900000,72.000,66.00,...,13311,fp_20cb129c3a,0.146062,NaN,6.500,6.500,"(65, 70]","(6, 8]",-0.50,6.500
1,2024-12-16,65.40,70.58,63.94,65.50,44152334,67.557143,67.366667,69.715,64.32,...,13311,fp_20cb129c3a,0.146062,1.0,4.315,4.315,"(65, 70]","(4, 6]",1.08,4.315
2,2024-12-09,66.52,68.66,64.80,65.96,26548753,67.328980,67.054074,71.300,64.59,...,13311,fp_20cb129c3a,0.146062,NaN,4.780,4.780,"(65, 70]","(4, 6]",1.93,4.780


In [11]:
df = df[df['difference_open_predicted_high']>0.1]
df.head(2)

,Date,Open,High,Low,Close,Volume,Close_EMA_13,Close_EMA_8,Predicted_High,Predicted_Low,...,total_tokens,system_fingerprint,combined_score,trade_tracking,difference_open_predicted,predicted_diff,open_bin,diff_bin,difference_open_predicted_low,difference_open_predicted_high
0,2024-12-23,65.5,67.92,65.45,67.9,8302204,67.900000,67.900000,72.000,66.00,...,13311,fp_20cb129c3a,0.146062,NaN,6.500,6.500,"(65, 70]","(6, 8]",-0.50,6.500
1,2024-12-16,65.4,70.58,63.94,65.5,44152334,67.557143,67.366667,69.715,64.32,...,13311,fp_20cb129c3a,0.146062,1.0,4.315,4.315,"(65, 70]","(4, 6]",1.08,4.315


In [12]:
# A) Frequency-based approach

import pandas as pd
import numpy as np

# Suppose df is your DataFrame
# Columns: ['Date', 'Open', 'Predicted_High', 'trade_result', ...]

df['predicted_diff_high'] = df['Predicted_High'] - df['Open']

# 1. Define bins
open_bins = [60, 65, 70, 75, 80, 85]    # Example
diff_bins = [0, 2, 4, 6, 8, 10]            # Example

# 2. Cut / bin the data
df['open_bin'] = pd.cut(df['Open'], bins=open_bins)
df['diff_bin'] = pd.cut(df['predicted_diff_high'], bins=diff_bins)

# 3. Group by (open_bin, diff_bin) and compute success probability
grouped = df.groupby(['open_bin', 'diff_bin'])
stats = grouped.agg(
    total_trades = ('trade_result', 'size'),
    success_trades = ('trade_result', 'sum')
).reset_index()

stats['success_rate'] = stats['success_trades'] / stats['total_trades']

# stats now has the empirical success rate for each bin
# e.g., if open_bin=(60,70], diff_bin=(4,6], success_rate=0.75 means 75% success in that bin


C:\Users\z0040jeb\AppData\Local\Temp\ipykernel_17560\668820209.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(['open_bin', 'diff_bin'])


In [13]:
stats.sort_values(by=['success_rate','open_bin'],ascending=False)

,open_bin,diff_bin,total_trades,success_trades,success_rate
20,"(80, 85]","(0, 2]",1,1,1.00
16,"(75, 80]","(2, 4]",2,2,1.00
10,"(70, 75]","(0, 2]",10,8,0.80
6,"(65, 70]","(2, 4]",5,4,0.80
15,"(75, 80]","(0, 2]",4,3,0.75
11,"(70, 75]","(2, 4]",8,4,0.50
7,"(65, 70]","(4, 6]",4,2,0.50
17,"(75, 80]","(4, 6]",2,0,0.00
18,"(75, 80]","(6, 8]",1,0,0.00
12,"(70, 75]","(4, 6]",3,0,0.00


In [42]:
import pandas as pd
import numpy as np

# 1. Define the success column:
df['predicted_low_success'] = df['prediction_deviation_Low'].apply(lambda x: 1 if x > 0 else 0)
df['predicted_diff_low'] = + df['Predicted_Low'] - df['Open']

# 2. Define your bins. 
#    Adjust these based on your dataset’s actual ranges
open_bins = [60, 65, 70, 75, 80, 85]           # Example bins for 'Open'
low_dev_bins = [-2, 0, 2, 4, 6, 8]            # Example bins for 'prediction_deviation_Low'

# 3. Cut / bin the data
df['open_bin'] = pd.cut(df['Open'], bins=open_bins)
df['low_dev_bin'] = pd.cut(df['predicted_diff_low'], bins=low_dev_bins)

# 4. Group by your bins and aggregate
grouped = df.groupby(['open_bin', 'low_dev_bin'])
stats = grouped.agg(
    total_trades=('predicted_low_success', 'size'),
    success_trades=('predicted_low_success', 'sum')
).reset_index()

# 5. Compute success rate
stats['success_rate'] = stats['success_trades'] / stats['total_trades']

# 6. stats DataFrame now contains your success rate in each (open_bin, low_dev_bin) bucket.
stats.sort_values(by='success_rate',ascending=False)


C:\Users\z0040jeb\AppData\Local\Temp\ipykernel_17560\2706613805.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(['open_bin', 'low_dev_bin'])


,open_bin,low_dev_bin,total_trades,success_trades,success_rate
1,"(60, 65]","(0, 2]",2,2,1.000000
6,"(65, 70]","(0, 2]",4,4,1.000000
16,"(75, 80]","(0, 2]",1,1,1.000000
7,"(65, 70]","(2, 4]",1,1,1.000000
10,"(70, 75]","(-2, 0]",8,6,0.750000
5,"(65, 70]","(-2, 0]",7,5,0.714286
0,"(60, 65]","(-2, 0]",0,0,NaN
2,"(60, 65]","(2, 4]",0,0,NaN
3,"(60, 65]","(4, 6]",0,0,NaN
4,"(60, 65]","(6, 8]",0,0,NaN


In [43]:
df[['prediction_deviation_Low','Predicted_Low','Low','Open','predicted_low_success']]


,prediction_deviation_Low,Predicted_Low,Low,Open,predicted_low_success
0,0.5500,66.000,65.4500,65.500,1
1,0.3800,64.320,63.9400,65.400,1
2,-0.2100,64.590,64.8000,66.520,0
3,2.8550,68.000,65.1450,71.000,1
4,-2.5160,66.620,69.1360,71.160,0
5,1.3600,65.680,64.3200,64.480,1
6,4.4100,69.000,64.5900,71.000,1
7,-2.8000,65.680,68.4800,70.250,0
8,2.3800,69.000,66.6200,67.295,1
9,2.7100,68.390,65.6800,68.100,1


In [21]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_EMA_13',
       'Close_EMA_8', 'Predicted_High', 'Predicted_Low', 'trade_result',
       'prediction_deviation_High', 'profit', 'profit_(10000)',
       'prediction_deviation_Low', 'ticker', 'train_window', 'target_window',
       'indicators', 'timeframe', 'training_data', 'Profitable_trades_(%)',
       'Total_win_trades', 'cumulative_profit', 'High_Mean_Negative_Deviation',
       'Low_Mean_Negative_Deviation', 'model', 'total_tokens',
       'system_fingerprint', 'combined_score', 'trade_tracking',
       'difference_open_predicted', 'predicted_diff', 'open_bin', 'diff_bin',
       'difference_open_predicted_low', 'difference_open_predicted_high',
       'predicted_diff_high', 'predicted_diff_low'],
      dtype='object')

In [24]:
df[['Low','Predicted_Low','prediction_deviation_Low']]
df['predicted_low_success'] = df['prediction_deviation_Low'].apply(lambda x: 1 if x> 0 else 0)
df

,Date,Open,High,Low,Close,Volume,Close_EMA_13,Close_EMA_8,Predicted_High,Predicted_Low,...,trade_tracking,difference_open_predicted,predicted_diff,open_bin,diff_bin,difference_open_predicted_low,difference_open_predicted_high,predicted_diff_high,predicted_diff_low,predicted_low_success
0,2024-12-23,65.500,67.9200,65.4500,67.90,8302204,67.900000,67.900000,72.000,66.000,...,NaN,6.500,6.500,"(65, 70]","(0.0, 2.0]",-0.500,6.500,6.500,0.500,1
1,2024-12-16,65.400,70.5800,63.9400,65.50,44152334,67.557143,67.366667,69.715,64.320,...,1.0,4.315,4.315,"(65, 70]",NaN,1.080,4.315,4.315,-1.080,1
2,2024-12-09,66.520,68.6600,64.8000,65.96,26548753,67.328980,67.054074,71.300,64.590,...,NaN,4.780,4.780,"(65, 70]",NaN,1.930,4.780,4.780,-1.930,0
3,2024-12-02,71.000,74.5200,65.1450,66.52,42449346,67.213411,66.935391,73.000,68.000,...,1.0,2.000,2.000,"(70, 75]",NaN,3.000,2.000,2.000,-3.000,1
4,2024-11-25,71.160,73.4900,69.1360,71.12,18337519,67.771495,67.865304,71.300,66.620,...,1.0,0.140,0.140,"(70, 75]",NaN,4.540,0.140,0.140,-4.540,0
5,2024-11-18,64.480,69.7150,64.3200,69.62,26033143,68.035567,68.255236,71.250,65.680,...,2.0,6.770,6.770,"(60, 65]","(0.0, 2.0]",-1.200,6.770,6.770,1.200,1
6,2024-11-11,71.000,71.3000,64.5900,64.76,29813463,67.567629,67.478517,71.640,69.000,...,3.0,0.640,0.640,"(70, 75]",NaN,2.000,0.640,0.640,-2.000,1
7,2024-11-04,70.250,73.4634,68.4800,70.41,26570203,67.973682,68.129958,72.980,65.680,...,1.0,2.730,2.730,"(70, 75]",NaN,4.570,2.730,2.730,-4.570,0
8,2024-10-28,67.295,76.0600,66.6200,70.99,48524182,68.404585,68.765523,73.000,69.000,...,1.0,5.705,5.705,"(65, 70]","(0.0, 2.0]",-1.705,5.705,5.705,1.705,1
9,2024-10-21,68.100,72.9800,65.6800,71.25,41260416,68.811073,69.317629,71.640,68.390,...,1.0,3.540,3.540,"(65, 70]","(0.0, 2.0]",-0.290,3.540,3.540,0.290,1
